# Dependency Injection in FastAPI

- authentication system
- query parameters' validator
- rate-limiter

In FastAPI a dependency injection can even call another one recursively

## What is dependency injection?

Generally speaking, dependency injection is a system able to automatically instantiate objects and the ones they depend on.

In [1]:
from fastapi import FastAPI, Header

app = FastAPI()


@app.get("/")
async def header(user_agent: str = Header(...)):
    return {
        "user_agent": user_agent
    }

In [3]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

In [4]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [38307]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60576 - "POST /posts HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:60580 - "POST / HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:60580 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:60617 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60617 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60618 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [38307]


![uk6VMU](https://raw.githubusercontent.com/azataiot/images/master/2021/12/20/uk6VMU.png)

Internally, the Header function has some logic to automatically get the request object, check for the required header, return its value, or raise an error if it's not present. From the developer's perspective, however, we don't know how it handled the required objects for this operation: we just ask for the value we need. That's dependency injection.

## Creating and using a function dependency
In FastAPI, a dependency can be defined either as a function or as a callable class

In [5]:
from fastapi import Depends

app = FastAPI()
from typing import Tuple


async def pagination(skip: int = 0, limit: int = 10) -> Tuple[int, int]:
    return skip, limit


@app.get("/items")
async def list_items(p: Tuple[int, int] = Depends(pagination)):
    skip, limit = p
    return {
        "skip": skip,
        "limit": limit
    }

In [6]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [38307]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60932 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:60932 - "GET /items/ HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:60932 - "GET /items HTTP/1.1" 200 OK
INFO:     127.0.0.1:60938 - "GET /items/?skip=5 HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:60938 - "GET /items?skip=5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:60944 - "GET /items/?limit=30&skip=5 HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:60944 - "GET /items?limit=30&skip=5 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [38307]


![CleanShot2021-12-20at18.50.00](https://raw.githubusercontent.com/azataiot/images/master/2021/12/20/CleanShot%202021-12-20%20at%2018.50.00.png)

Actually dependency is also for resuing. as soon as we have the same filters on other pages of the website, we can just pass the same dependency and that is all, we do not have to repeat ourselves then.


In [4]:
from fastapi import FastAPI, Depends

app = FastAPI()
from typing import Tuple


async def pagination(skip: int = 0, limit: int = 10) -> Tuple[int, int]:
    return skip, limit


@app.get("/items")
async def list_items(p: Tuple[int, int] = Depends(pagination)):
    skip, limit = p
    return {
        "skip": skip,
        'limit': limit
    }


@app.get("/settings")
async def list_settings(p: Tuple[int, int] = Depends(pagination)):
    skip, limit = p
    return {
        "skip": skip,
        'limit': limit
    }

In [5]:
from fastapi import Query


async def pagination(
        skip: int = Query(0, ge=0),
        # default value is 0 , ge=0 also makes sure that we only receive tha positive integers.
        limit: int = Query(10, ge=0),  # default value is 10
) -> Tuple[int, int]:
    capped_limit = min(100, limit)  # the maximum value for limit is 100 now.
    return skip, capped_limit

### Get an Object or raise a 404 error

same logic for :

- get
- update
- delete

try to retrieve tha object in the database or raise an error 404 if it dose not exist.

In [6]:
from fastapi import HTTPException, status


class Post:
    pass


db = {}


async def get_post_or_404(uid: int) -> Post:
    try:
        return db.posts[id]
    except KeyError:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND)

Another typical example for the Dependency is authentication.

In [ ]:
app = FastAPI()


class PostUpdate:
    pass


@app.get("posts/{post_id}")
async def get_post(post: Post = Depends(get_post_or_404)):
    return Post


@app.patch("posts/{post_id}")
async def update_post(post_update: PostUpdate, post: Post = Depends(get_post_or_404)):
    updated_post = post.copy(update=post_update.dict())
    db.posts[post.id] = updated_post
    return updated_post


@app.delete("/posts/{id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_post(post: Post = Depends(get_post_or_404)):
    db.posts.pop(post.id)

## Creating and using a parameterized dependency with a class

In [3]:
from fastapi import FastAPI, Query, Depends
from typing import Tuple


class Pagination:
    def __init__(self, maximum_limit: int = 100):
        self.maximum_limit = maximum_limit

    async def __call__(self,
                       skip: int = Query(0, ge=0),
                       limit: int = Query(10, ge=0)
                       ) -> Tuple[int, int]:
        capped_limit = min(self.maximum_limit, limit)
        return skip, capped_limit

In [4]:
pagination = Pagination(maximum_limit=50)

In [5]:


app = FastAPI()


@app.get("/items")
async def list_items(p: Tuple[int, int] = Depends(pagination)):
    skip, limit = p
    return {
        "skip": skip,
        'limit': limit
    }

Use class methods as dependencies


In [6]:
class Pagination:
    def __init__(self, maximum_limit: int = 100):
        self.maximum_limit = maximum_limit

    async def skip_limit(self,
                         skip: int = Query(0, ge=0),
                         limit: int = Query(10, ge=0)
                         ) -> Tuple[int, int]:
        capped_limit = min(self.maximum_limit, limit)
        return skip, capped_limit

    async def page_size(self,
                        page: int = Query(1, ge=1),
                        size: int = Query(10, ge=10)
                        ) -> Tuple[int, int]:
        capped_size = min(self.maximum_limit, size)
        return page, capped_size

In [8]:
pagination = Pagination(maximum_limit=50)


@app.get("/items")
async def list_items(p: Tuple[int, int] = Depends(pagination.skip_limit)):
    skip, limit = p
    return {"skip": skip, "limit": limit}


@app.get("/things")
async def list_things(p: Tuple[int, int] = Depends(pagination.page_size)):
    page, size = p
    return {"page": page, "size": size}

## Using dependency at a path, router, and global level

In [9]:
from typing import Optional, Tuple
from fastapi import FastAPI, Depends, Header, Query


def secret_header(secret_header: Optional[str] = Header(None)) -> None:
    if not secret_header or secret_header != "SECRET_VALUE":
        raise HTTPException(status.HTTP_403_FORBIDDEN)

### Use dependency on a path decorator

In [10]:
app = FastAPI()


@app.get("/protected-route", dependencies=[Depends(secret_header)])
async def protected_route():
    return {
        "hello": "world!"
    }

### Use a dependency on a whole route

In [ ]:
from fastapi import APIRouter

app = FastAPI()

router = APIRouter(dependencies=[Depends(secret_header)])


@router.get("/r1")
async def r1():
    return {}


@router.get("/r2")
async def r2():
    return {}


app.include_router(router, prefix="/router")

OR

In [11]:
from fastapi import APIRouter

app = FastAPI()

router = APIRouter()


@router.get("/r1")
async def r1():
    return {}


@router.get("/r2")
async def r2():
    return {}


app.include_router(router, prefix="/router",dependencies=[Depends(secret_header)])

### Use a dependency in a whole application

In [12]:
app = FastAPI(dependencies=[Depends(secret_header)])

![CleanShot2021-12-22at14.37.46](https://raw.githubusercontent.com/azataiot/images/master/2021/12/22/CleanShot%202021-12-22%20at%2014.37.46.png)